In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from classic_clustering import ClassicClustering
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import preprocessing

# Fazendo os pré-processamentos necessários

In [2]:
#abrindo o csv com os arquivos
df = pd.read_csv('Data.csv',sep='|',encoding='utf8')
textos = list(df['textos'])

In [3]:
#inicializo um objeto da classe que me ajuda com o pré-processamento
cc = ClassicClustering()
#defino as stopwords padrão da classe
cc.define_stop_words()

In [4]:
%%time
#recodifico tudo em utf-8, retiro caracteres especiais e converto tudo para letra minúscula
textos1 = [cc.limpa_utf8(texto.lower()) for texto in textos]

CPU times: user 7.35 s, sys: 135 ms, total: 7.48 s
Wall time: 7.5 s


In [5]:
print(textos1[0][:1000])

portaria/ms no 08, de 10 de abril de 1987 (publicada em dou, de 28 de abril de 1987) o diretor da divisao nacional de vigilancia sanitaria de produtos saneantes domissanitarios, no uso de suas atribuicoes e considerando o risco oferecido pelas formulacoes alaclinas, quando apresentadas sob a forma de liquido premido ou para pulverizacao, que podem provocar serias lesoes em mucosas oculares ou do trato respiratorio, resolve: 1o proibir a fabricacao e comercializacao de saneantes domissanitarios fortemente alcalinos apresentados sob a forma de liquido premido (aerossol), ou liquido para pulverizacao, tais como produtos para limpeza de fornos e desincrustacao de gorduras, de uso domestico. § unico: executam-se do acima disposto os produtos para uso profissional, os quais, nesse caso, devem ser aplicadas por pessoa habilitada, devidamente protegido por roupas, mascaras, oculos e indumetaria apropriados. 2o para efeito da presente sao considerados produtos fortemente alcalinos aqueles conte

In [6]:
%%time
#retiro romanos e stopwords
textos2 = []
for texto in textos1:
    novo_texto = [cc.tira_stopwords_e_romanos(palavra) for palavra in texto.split()]
    textos2.append(' '.join(novo_texto))

CPU times: user 21 s, sys: 184 ms, total: 21.2 s
Wall time: 21.2 s


In [7]:
print(textos2[0][:1000])

portaria/ms  08,  10  abril  1987 (publicada  dou,  28  abril  1987)  diretor  divisao nacional  vigilancia sanitaria  produtos saneantes domissanitarios,  uso   atribuicoes  considerando  risco oferecido  formulacoes alaclinas,  apresentadas   forma  liquido premido   pulverizacao,  podem provocar serias lesoes  mucosas oculares   trato respiratorio, resolve: 1o proibir  fabricacao  comercializacao  saneantes domissanitarios fortemente alcalinos apresentados   forma  liquido premido (aerossol),  liquido  pulverizacao, tais  produtos  limpeza  fornos  desincrustacao  gorduras,  uso domestico.  unico: executam-se  acima disposto  produtos  uso profissional,  quais, nesse caso, devem  aplicadas  pessoa habilitada, devidamente protegido  roupas, mascaras, oculos  indumetaria apropriados. 2o  efeito  presente  considerados produtos fortemente alcalinos  contendo bases inorganicas livres  teores acima  1%, inclusive. 3o  produtos alcalinos, assim compreendidos  cuja medida  ph exceda, 8,5, 

In [8]:
%%time
cc.textos_tratados = textos2
cc.stem()
base_tfidf = cc.vec_tfidf()

Comecou a fazer o stemming.
Tempo para fazer o stemming: 118.23791909217834

CPU times: user 1min 59s, sys: 229 ms, total: 2min
Wall time: 2min


In [9]:
print(cc.textos_stem[0][:1000])

portaria/m 08, 10 abril 1987 (public dou, 28 abril 1987) dire divisa nacion vigilanc sanit produt sane domissanitarios, uso atribuico consider risc oferec formulaco alaclinas, apresent form liqu prem pulverizacao, pod provoc ser leso mucos ocul trat respiratorio, resolve: 1o proib fabricaca comercializaca sane domissanitari fort alcalin apresent form liqu prem (aerossol), liqu pulverizacao, tal produt limp forn desincrustaca gorduras, uso domestico. unico: executam-s acim dispost produt uso profissional, quais, ness caso, dev aplic pesso habilitada, devid proteg roupas, mascaras, ocul indumet apropriados. 2o efeit pres consider produt fort alcalin cont bas inorgan livr te acim 1%, inclusive. 3o produt alcalinos, assim compreend cuj med ph exceda, 8,5, fic abrang dispost artig 1o. 4o fic conced praz 90 (noventa) dias, fabric produt registr event dat anteri publicaca pres infrij disposi aqui, contidos, promov recolh unidad exist comercio, find qual mesm recolh fiscalizaca inutilizadas. a

In [10]:
%%time
n_dims = 600
X = cc.SVD(base_tfidf, n_dims)

Começou a redução de dimensionalidade.
Número de dimensões de entrada: 51330
600 dimensões explicam 0.8112234061058099 da variância.
Tempo para fazer a redução de dimensionalidade: 30.418655157089233

CPU times: user 1min, sys: 14.2 s, total: 1min 14s
Wall time: 30.4 s


# Agora vamos rodar os modelos

In [11]:
#gerando labels para as normas (sei que não é a forma mais esperta de se fazer isso)
macrotema_por_norma = list(df['macrotemas'])
macrotemas = list(dict.fromkeys(macrotema_por_norma))

di = dict.fromkeys(macrotema_por_norma)
i=0
for macrotema in macrotemas:
    di[macrotema] = i
    i+=1

y = np.zeros(len(macrotema_por_norma))
i=0
for m in macrotema_por_norma:
    y[i] = di[m]
    i+=1

In [23]:
#normalizando as features com média zero e std 1
#X = preprocessing.scale(X)

In [12]:
#separando em dados de treinamento e validação
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
models = [
    LogisticRegression(random_state=42, solver='lbfgs', multi_class='multinomial'),
    SVC(C=1,gamma='scale'),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    AdaBoostClassifier()]

In [14]:
for model in models:
    model.fit(X_train,y_train)
    print(model.score(X_test,y_test))

0.8111111111111111
0.8333333333333334
0.2361111111111111
0.4111111111111111


In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

pipe = Pipeline([('classifier' , SVC(random_state=0,decision_function_shape='ovo'))])

param_grid = [
    {'classifier' : [SVC()],
    'classifier__C' : np.logspace(-4, 4, 20), # valores de 0 a 10.000
    'classifier__kernel' : ['linear','rbf','poly','sigmoid']}]

# Create grid search object
clf = GridSearchCV(pipe, param_grid = param_grid, cv = 10, verbose=True, n_jobs=-1)
best_clf = clf.fit(X_train, y_train)

print("Acuracia na base de treino da LR com hiperparâmetros ajustados: ", clf.score(X_train, y_train))
print("Acuracia na base de teste da LR com hiperparâmetros ajustados: ", clf.score(X_test, y_test))

Fitting 10 folds for each of 80 candidates, totalling 800 fits


/usr/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   46.5s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 14.2min
[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed: 14.3min finished
/usr/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/usr/lib/python3.7/site-packages/sklearn/svm

Acuracia na base de treino da LR com hiperparâmetros ajustados:  0.9680555555555556
Acuracia na base de teste da LR com hiperparâmetros ajustados:  0.8666666666666667


In [27]:
#acessando os resultados do grid search
df = pd.DataFrame(clf.cv_results_)
#achando quais foram os parâmetros do melhor classificador
np.argmax(df['mean_test_score'])

/usr/lib/python3.7/site-packages/numpy/core/fromnumeric.py:61: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return bound(*args, **kwds)


71

In [28]:
#printando os parâmetros do melhor classificador
df.iloc[71,:]

mean_fit_time                                                         3.50312
std_fit_time                                                         0.317659
mean_score_time                                                      0.252879
std_score_time                                                      0.0585404
param_classifier            SVC(C=1438.44988828766, cache_size=200, class_...
param_classifier__C                                                   1438.45
param_classifier__kernel                                              sigmoid
params                      {'classifier': SVC(C=1438.44988828766, cache_s...
split0_test_score                                                    0.807947
split1_test_score                                                    0.854305
split2_test_score                                                    0.812081
split3_test_score                                                    0.898649
split4_test_score                                               